In [29]:
import os
import csv
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import Paragraph

def generate_autopsy_report(pdf_filename, case_info):
    # Create a PDF document
    c = canvas.Canvas(pdf_filename, pagesize=letter)

    # Add title
    c.setFont("Helvetica-Bold", 20)
    c.drawString(50, 750, "AI 05  Chronicles of the Corpse: AI's Murder Tale")

    # Add space between title and photo
    c.drawString(50, 700, "")  # Adjust y-coordinate for desired space

    # Add body image (smaller dimensions)
    body_image = "body_image.jpg"  # Path to your body image file
    c.drawImage(body_image, x=100, y=600, width=200, height=150)  # Adjust width and height as needed

    # Add space between photo and data blocks
    c.drawString(50, 550, "")  # Adjust y-coordinate for desired space

    # Add case information blocks
    styles = get_paragraph_styles()
    x_positions = [50, 300]  # Starting x positions for two columns
    y_position = 530  # Starting y position
    for key, value in case_info.items():
        paragraph = Paragraph(f"<b>{key}:</b> {value}", styles[key])
        width_needed, height_needed = paragraph.wrap(200, 200)  # Adjust wrap width and maximum height to fit the page
        c.rect(x_positions[0], y_position, 200, height_needed + 10)
        paragraph.drawOn(c, x_positions[0] + 10, y_position + 5)  # Center the paragraph in the block
        y_position -= height_needed + 30  # Increase the space between data blocks

    # Save the PDF
    c.save()

def get_paragraph_styles():
    styles = {}
    styles['Case Number'] = ParagraphStyle(name='Case Number', fontSize=12, leading=14)
    styles['Date of Death'] = ParagraphStyle(name='Date of Death', fontSize=12, leading=14)
    styles['Date of Incident'] = ParagraphStyle(name='Date of Incident', fontSize=12, leading=14)
    styles['Age'] = ParagraphStyle(name='Age', fontSize=12, leading=14)
    styles['Gender'] = ParagraphStyle(name='Gender', fontSize=12, leading=14)
    styles['Race'] = ParagraphStyle(name='Race', fontSize=12, leading=14)
    styles['Latino'] = ParagraphStyle(name='Latino', fontSize=12, leading=14)
    styles['Manner of Death'] = ParagraphStyle(name='Manner of Death', fontSize=12, leading=14)
    styles['Primary Cause'] = ParagraphStyle(name='Primary Cause', fontSize=12, leading=14)
    styles['Secondary Cause'] = ParagraphStyle(name='Secondary Cause', fontSize=12, leading=14)
    styles['Incident Address'] = ParagraphStyle(name='Incident Address', fontSize=12, leading=14)
    return styles

def process_csv(input_csv_file, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with open(input_csv_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for i, row in enumerate(reader):
            case_info = {
                'Case Number': row.get('Case Number', f'Case {i+1}'),  # Use 'Case X' if 'Case Number' column doesn't exist
                'Date of Death': row['Date of Death'],
                'Date of Incident': row['Date of Incident'],
                'Age': row['Age'],
                'Gender': row['Gender'],
                'Race': row['Race'],
                'Latino': row['Latino'],
                'Manner of Death': row['Manner of Death'],
                'Primary Cause': row['Primary Cause'],
                'Secondary Cause': row.get('Secondary Cause', ''),
                'Incident Address': row['Incident Address'],
            }
            pdf_filename = os.path.join(output_folder, f"autopsy_report_{i+1}.pdf")
            generate_autopsy_report(pdf_filename, case_info)

# Example usage:
process_csv("autopsy report new.csv", "autopsy_reports")


In [38]:
def analyze_pdf_folder(folder_path):
    all_case_info = []

    # Read data from PDF files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "rb") as f:
                pdf = SimpleDocTemplate(file_path)
                elements = pdf.getPage(0).extractText()

            # Extract the case information
            case_info = {}
            for line in elements.split("\n"):
                key, value = line.split(":", 1)
                case_info[key.strip()] = value.strip()

            all_case_info.append(case_info)

    # Determine the common causes
    common_causes = {
        "Primary Cause": {"gun": 0, "knife": 0, "poison": 0, "hanging": 0, "drugged": 0, "stabbed": 0},
        "Secondary Cause": {"gun": 0, "knife": 0, "poison": 0, "hanging": 0, "drugged": 0, "stabbed": 0},
    }

    for case_info in all_case_info:
        for cause_type in common_causes:
            for cause in common_causes[cause_type]:
                if cause in case_info[cause_type]:
                    common_causes[cause_type][cause] += 1

    return all_case_info, common_causes
def generate_summary_report(summary_filename, case_info_list, common_causes):

    # Determine the number of cases
    num_cases = len(case_info_list)

    # Calculate percentages
    for cause_type in common_causes:
        total = sum(common_causes[cause_type].values())
        for cause in common_causes[cause_type]:
            common_causes[cause_type][cause] = common_causes[cause_type][cause] / total * 100

    # Analyze and create assumptions
    assumptions = []

    for cause_type in common_causes:
        for cause, percentage in common_causes[cause_type].items():
            if percentage > 50:
                assumptions.append(f"There might be a prevalence of {cause.capitalize()} in the primary causes.")
                assumptions.append(f"There might be a prevalence of {cause.capitalize()} in the secondary causes.")

    content = [
        ["Number of Cases", num_cases],
        ["Top Primary Cause", max(common_causes["Primary Cause"], key=common_causes["Primary Cause"].get)],
        ["Top Secondary Cause", max(common_causes["Secondary Cause"], key=common_causes["Secondary Cause"].get)],
        *assumptions,
    ]

    # Create the summary report
    table_style = [
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 0), (-1, 0), 14),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 12),
        ("BACKGROUND", (0, 0), (-1, 0), colors.grey),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTSIZE", (0, 1), (-1, -1), 12),
        ("GRID", (0, 0), (-1, -1), 1, colors.black),
    ]

    table_data = Table(content)
    table_data.setStyle(TableStyle(table_style))

    doc = SimpleDocTemplate(summary_filename, pagesize=letter)
    doc.build([Paragraph("Summary of Analyzed Cases", ParagraphStyle("Heading2"))] + [table_data])

# Example usage
case_info_list, common_causes = analyze_pdf_folder("autopsy_reports")
generate_summary_report("summary_report.pdf", case_info_list, common_causes)

NameError: name 'SimpleDocTemplate' is not defined

In [39]:
def analyze_pdf_folder(folder_path):
    all_case_info = []

    # Read data from PDF files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "rb") as f:
                c = canvas.Canvas(file_path)
                elements = c.getPage(0).extractText()

            # Extract the case information
            case_info = {}
            for line in elements.split("\n"):
                key, value = line.split(":", 1)
                case_info[key.strip()] = value.strip()

            all_case_info.append(case_info)

    # Determine the common causes
    common_causes = {
        "Primary Cause": {"gun": 0, "knife": 0, "poison": 0, "hanging": 0, "drugged": 0, "stabbed": 0},
        "Secondary Cause": {"gun": 0, "knife": 0, "poison": 0, "hanging": 0, "drugged": 0, "stabbed": 0},
    }

    for case_info in all_case_info:
        for cause_type in common_causes:
            for cause in common_causes[cause_type]:
                if cause in case_info[cause_type]:
                    common_causes[cause_type][cause] += 1

    return all_case_info, common_causes